In [1]:
// TypeScript Jupyter extension
import * as tslab from "tslab";

// CSC 600 Libraries
import { requireCytoscape, requireCarbon, drawTree } from "./lib/draw";
import { mkLeaf, mkLeafNode, mkNode } from "./lib/tree"

requireCarbon();
requireCytoscape();

# Recursion

## Where Were We?

1. **Language primitives** (i.e., building blocks of languages)
    * Last time: sum types
    * This time: **recursion**
2. Language paradigms (i.e., combinations of language primitives)
3. Building a language (i.e., designing your own language)

## Goal

1. Introduce the concept of **recursive functions**.
2. Highlight **recursive** thinking as a problem solving method.
3. We will see that iteration and recursion are equivalent. But recursion may be a better technique for certain problems.

## Outline

- Iteration vs. recursion via example

## Iteration vs. Recursion

Let's just look at a problem ...

### Example 1: Smallest Number in Array

Given an array of numbers, find the smallest element in the array.

In [2]:
// Example 1
const xs = [10000000000];
console.log("Smallest", 10000000000);

Smallest 10000000000


In [3]:
// Example 2
const xs = [2, 1];
console.log("Smallest", 1);

Smallest 1


In [4]:
// Example 3
const xs = [];
console.log("Smallest", undefined);

Smallest undefined


In [5]:
// Example 4
const xs = [32, 21, 13, 90];
console.log("Smallest", 13);

Smallest 13


### An iterative solution

I know most of you can code this in your sleep ...

In [6]:
function iterFindSmallest(xs: number[]): number | undefined {
    if (xs.length === 0) {
        return undefined;
    }
    
    let smallest = xs[0];
//     for (let i = xs.length - 1; i >= 0; i--) {
//         smallest = Math.min(smallest, xs[i]);
//     }
    for (let i=1; i < xs.length; i++) {  // <- iterative solution
        smallest = Math.min(smallest, xs[i]);
    }
    return smallest;
}

In [7]:
const xs = [32, 21, 13, 90]
iterFindSmallest(xs);

13


#### This is a perfectly fine solution.

* The code is clean.
* It uses sum types to express that the smallest number in an empty array is `undefined` as opposed to having the user guess what might happen.
* The function is a pure function.

#### However the solution requires "operational" thinking.

Operational thinking means: executing the code line-by-line mentally. This raises several questions
* Do I have to traverse the array from front to back?
* What happens if I traverse the array in a different order?
* What happens if I use parallel or distributed computing?
* How could I argue that this solution is correct? (In particular, we might wonder what the *loop invariant* is.)

### Another solution?

We might wonder if there is another way that makes it apparent that the code is correct.

### Observation

```ts
smallest([a, b, c, d]) = min(a, smallest([b, c, d]))
                       = min(a, min(b, smallest([c, d])))
                       = min(a, min(b, min(c, smallest([d]))))
                       = min(a, min(b, min(c, d))))
```
where
1. `smallest` is some hypothetical function that gives the smallest of an array of number.
2. `min` returns the smallest of two numbers.
3. we can think of this as a series of equations that needs to be satisfied

In [8]:
const tr = mkNode("smallest([a, b, c, d])",
                  mkLeaf(),
                  mkNode("smallest([b, c, d])",
                         mkLeaf(),
                         mkNode("smallest([c, d])",
                                mkLeaf(),
                                mkLeafNode("smallest([d])"))));
drawTree(tr);

#### Key Properties

```ts
smallest([a, b, c, d]) = min(a, smallest([b, c, d]))                (recursive call 1)
                       = min(a, min(b, smallest([c, d])))           (recursive call 2)
                       = min(a, min(b, min(c, smallest([d]))))      (recursive call 3)
                       = min(a, min(b, min(c, d))))                 (base case)
```
1. We have reduced the construction of smallest into a nested sequences of call to `min`.
2. `min` should have an implementation
```ts
function min(a: number, b: number): number { return a < b ? a : b; }
```
3. There are *recursive* cases where an equality involves `smallest` on both sides.
4. There is a *base* case `smallest([d])` that reduces to an implementation without reference to `smallest`.

## Recursion

We can use **recursion** to solve a problem whenever we can use a solution to a **smaller instance** of the **same problem** to solve the original problem. For example,

```ts
smallest([a, b, c, d]) = min(a, smallest([b, c, d]))
```

means that the solution to `smallest([a, b, c, d])` can be given in terms of `smallest([b, c, d])` where `[b, c, d]` is the smaller instance of the array `[a, b, c, d]`.

### Let's code this up

```ts
smallest([a, b, c, d]) = min(a, smallest([b, c, d]))                (recursive call 1)
                       = min(a, min(b, smallest([c, d])))           (recursive call 2)
                       = min(a, min(b, min(c, smallest([d]))))      (recursive call 3)
                       = min(a, min(b, min(c, d))))                 (base case)
```

Generalizing from the above

```ts
smallest([]) = undefined
smallest([x]) = x
smallest([a, b, c, ...]) = min(a, smallest([b, c, ...]))            (recursive case)
```

In [9]:
// Aside: slicing
console.log("original array", xs);
console.log(xs.slice(1));    // Without the first element
console.log(xs.slice(2));    // Without the first two elements
console.log(xs.slice(1,3));  // Elements 1 through 2

original array [ 32, 21, 13, 90 ]
[ 21, 13, 90 ]
[ 13, 90 ]
[ 21, 13 ]


In [10]:
function recFindSmallest1(xs: number[]): number | undefined {
    if (xs.length === 0) {
        // smallest([]) = undefined
        return undefined;
    } else if (xs.length === 1) {
        // smallest([x]) = x
        return xs[0];
    } else {
        // smallest([a, b, c, ...]) = min(a, smallest([b, c, ...]))
        return Math.min(xs[0], recFindSmallest1(xs.slice(1)));
    }
}

In [11]:
const xs = [32, 21, 13, 90];
console.log("iterative", iterFindSmallest(xs));
console.log("recursive", recFindSmallest1(xs));

iterative 13
recursive 13


### Aside: Differential Testing

Now that we have two different implementations of the same function, we can do something called *differential testing* where we compare the input/output behavior of the two functions. You can use this to increase confidence that your functions are correct
1. If the outputs are the same, high probability that the functions are correct.
2. If the outputs are different, at least one of the functions is incorrect.

One application: test an optimized vs. an unoptimized function

In [12]:
console.log(iterFindSmallest([]) === recFindSmallest1([]));
console.log(iterFindSmallest([32, 13]) === recFindSmallest1([32, 13]));

true
true


### Another recursive solution?

```ts
smallest([a, b, c, d]) = min(smallest([a, b]), smallest([c, d]))                                 (recursive call 1)
                       = min(min(smallest([a]), smallest[b]), min(smallest([c]), smallest[d])))  (recursive call 2)
                       = min(min(a, b), min(c, d)))                                              (base case)
```

In [13]:
const tr = mkNode("smallest([a, b, c, d])",
                  mkNode("smallest([a, b])",
                         mkLeafNode("smallest([a])"),
                         mkLeafNode("smallest([b])")),
                  mkNode("smallest([c, d])",
                         mkLeafNode("smallest([c])"),
                        mkLeafNode("smallest([d])")));
drawTree(tr);

In [14]:
function recFindSmallest2(xs: number[]): number | undefined {
    if (xs.length === 0) {
        return undefined;
    } else if (xs.length === 1) {
        return xs[0];
    } else {
        const midpt = Math.floor(xs.length / 2);
        return Math.min(recFindSmallest2(xs.slice(0, midpt)), recFindSmallest2(xs.slice(midpt)));
    }
}

recFindSmallest2([32, 21, 13, 90]);

13


### Aside: Differential Testing 2

- We can still apply the idea of differential testing if we have three implementations.
- This time, we can vote on the most likely outcome.
- If they all agree, then there is a high probability that the answer is correct.
- If two agree, then the one that disagrees is likely the one that is wrong.
- If they all disagree, then there are at least two functions that are wrong.

In [15]:
const funcs = [iterFindSmallest, recFindSmallest1, recFindSmallest2];
const inputs = [
    [],
    [32, 11, 13, 15],
    [32, 11, 13, -15],
]

const outputs = inputs.map((x) => funcs.map((fn) => fn(x)))
for (const output of outputs) {
    console.log(output)
}

[ undefined, undefined, undefined ]
[ 11, 11, 11 ]
[ -15, -15, -15 ]


### Example 2: Sorting

- Given an array of numbers, sort them into ascending order.
- Unlike finding the smallest number in an array, might be harder to code.

### Insertion Sort

```ts
sortedUnsorted([], [13, 32, -1, 10])
    = sortedUnsorted(insertInOrder([], 13), [32, -1, 10])     (insert 13 in order)
    = sortedUnsorted(insertInOrder([13], 32), [-1, 10])     (insert 32 in order)
    = sortedUnsorted(insertInOrder([13, 32], -1), [10])     (insert -1 in order)
    = sortedUnsorted(insertInOrder([-1, 13, 32], 10), [])   (insert 10 in order)
```

In [16]:
// Here's a "recursive"-looking insertion sort
function insertionSort(arr: number[]): number[] {  // Question: pure or impure?
    function insertInOrder(x: number, sorted: number[]): number[] {  // Question: pure or impure?
        for (let i = 0; i < sorted.length; i++) {
            if (x < sorted[i]) {
                sorted.splice(i, 0, x);
                return sorted;
            }
        }
        sorted.push(x);
        return sorted;
    }
    
    function sortedUnsorted(sorted: number[], unsorted: number[]): number[] {  // Question: pure or impure?
        if (unsorted.length === 0) {
            return sorted;
        } else {
            return sortedUnsorted(insertInOrder(unsorted[0], sorted), unsorted.slice(1));
        }
    }
    
    return sortedUnsorted([], arr);
}

In [17]:
console.log(insertionSort([13, 32, -1, 10]));
console.log(insertionSort([13, 32, -1, 10, -2]));

[ -1, 10, 13, 32 ]
[ -2, -1, 10, 13, 32 ]


## Insertion and Recursion are "Equally Expressive"

- One may wonder if iteration or recursion is "better".
- They are equally expressive.
- However, iteration may take less memory because each recursive call requires an additional stack frame.
- Recursion may make it easier to see why certain complex loops work (e.g., rederiving insertion sort).

### Iteration to Recursion

#### Example

In [18]:
let acc = 0;
for (let i = 0; i < 10; i++) {
    acc += i;
}
acc

45


In [19]:
let acc = 0;
function recForLoop1(i: number): void {  // Challenge: how would you make this a pure function?
    if (i < 10) {
        acc += i;
        recForLoop1(i + 1);
    } 
}

recForLoop1(0);
acc;

45


#### In General

In [20]:
function recForLoop(init: () => void, test: () => boolean, update: () => void, body: () => any): any {
    init();
    function go() {
        if (test()) {
            body();
            update();
            go();
        }
    }
    go();
}

In [21]:
let i;
let acc = 0;
function init(): void {
    i = 0;
}
function test(): boolean {
    return i < 10;
}
function update(): void {
    i++;
}
function body(): void {
    acc += i;
}

// Question: what are init, test, update, body?
recForLoop(init, test, update, body);
acc

45


### Recursion to Iteration

In [22]:
function factorial(n: number): number {
    if (n === 0) {
        return 1;
    } else {
        return n * factorial(n - 1);
    }
}

In [23]:
function iterFactorial(n: number): number {
    const stack: [number, "CALL"|"RET"][] = [[n, "CALL"]];
    
    while (stack.length > 0) {
        const [n, mode]: [number, "CALL"|"RET"] = stack[stack.length-1];
        if (mode === "CALL") {
            if (n === 0) {
                stack.pop();
                stack.push([1, "RET"]);
            } else {
                stack.push([n - 1, "CALL"]);
            }
        } else {
            stack.pop();
            if (stack.length === 0) {
                return n;
            }
            const [m, mode]: [number, "CALL"|"RET"] = stack.pop();
            stack.push([n * m, "RET"]);
        }
    }
}

In [24]:
console.log(factorial(4), iterFactorial(4));
console.log(factorial(5), iterFactorial(5));

24 24
120 120


### In general, we need to simulate the call stack!

## Summary

- We saw that recursion was a way to solve a problem by solving smaller instances of the same problem.
- Iteration and recursion are equivalent, i.e., we can express the same computations.
- Iteration may take up less memory because we do not need a call stack.
- Recursion may make it easier to reason about why certain algorithms work.